## Build ETLs for datasets

In [1]:
import requests as r
import pandas as pd
from urllib.request import urlopen
from zipfile import ZipFile

In [3]:
#create the ETL to download the risk score shapefile from FEMA.gov
    
base = "https://nri-data-downloads.s3.amazonaws.com/"
    
countyfile = "NRI_Shapefile_Counties.zip"


In [4]:
NRI = str(base + countyfile)
print(NRI)

https://nri-data-downloads.s3.amazonaws.com/NRI_Shapefile_Counties.zip


In [5]:
r.get(NRI)


<Response [200]>

In [15]:
nriData = r.get(NRI)

In [17]:
with open("/Users/celia/Desktop/NRI_Shapefile_Counties.zip", "wb") as zip:
    zip.write(nriData.content)

In [18]:
with zipfile.ZipFile("/Users/celia/Desktop/NRI_Shapefile_Counties.zip","r") as zip_ref:
    zip_ref.extractall()

In [19]:
#create the ETL to download the USGS Renewable Energy Potential shapefile
#I couldn't get this ETL to work, and it was starting to become a large chunk of time, so I had to stop
#I learned I am still not strong at json-based ETLs and will need to practice with them

base5 = 'https://gds-api.nrel.gov/oc-backend/api/download_proxy/re-atlas/'
solar = 'BYFsT7/ReAtlas_us_csp_total.zip'

file:///C:/Users/celia/AppData/Local/Temp/ReAtlas_us_csp_total.zip

In [23]:
solarfile = str(base5 + solar)
print(solarfile)

file:///C:/Users/celia/AppData/Local/Temp/ReAtlas_us_csp_total.zip


In [ ]:
solarFile = r.get(solarfile)

## Make Suitability Layer

In [3]:
#making a suitability analysis layer from the High Need Counties shapefile of 636 counties in need of electrical infrastructure
arcpy.ba.MakeSuitabilityAnalysisLayer("High_Need_Counties", "Suit_Analysis3")

<Result 'Suit_Analysis3'>

## Run Analysis

In [5]:
#add variable from Esri demographics
arcpy.ba.AddVariableBasedSuitabilityCriteria("Suit_Analysis3", "populationtotals.popgrwcyfy;householdincome.pci_cy;policy.white_cy;policy.black_cy;policy.amerind_cy;policy.asian_cy;policy.pacific_cy;policy.othrace_cy;policy.race2up_cy")

id,value
0,a Layer object
1,populationtotals.popgrwcyfy;householdincome.pci_cy;policy.white_cy;policy.black_cy;policy.amerind_cy;policy.asian_cy;policy.pacific_cy;policy.othrace_cy;policy.race2up_cy


In [2]:
#convert polygon layers to points
arcpy.management.FeatureToPoint("High_Need_counties_solar_Spa", "solar_point")
arcpy.management.FeatureToPoint("wind_potential", "wind_point")
arcpy.management.FeatureToPoint("geo_potential", "geo_point")

<Result 'C:\\Users\\celia\\Documents\\ArcGIS\\Projects\\GIS5572_FinalProj\\GIS5572_FinalProj.gdb\\geo_point'>

In [7]:
#add the points to the suitability criteria
arcpy.ba.AddPointLayerBasedSuitabilityCriteria("Suit_Analysis3", "GISJOIN", "solar_point", "WEIGHT", None, None, None, None, "ave_dni", "MAX", None)

id,value
0,a Layer object
1,PointWeight_MAX


In [8]:
arcpy.ba.AddPointLayerBasedSuitabilityCriteria("Suit_Analysis3", "GISJOIN", "geo_point", "WEIGHT", None, None, None, None, "class", "MAX", None)

id,value
0,a Layer object
1,PointWeight_MAX0


In [9]:
arcpy.ba.AddPointLayerBasedSuitabilityCriteria("Suit_Analysis3", "GISJOIN", "wind_point", "WEIGHT", None, None, None, None, "a30", "MAX", None)

id,value
0,a Layer object
1,PointWeight_MAX1


In [10]:
arcpy.ba.AddPointLayerBasedSuitabilityCriteria("Suit_Analysis3", "GISJOIN", "NRI_Point", "WEIGHT", None, None, None, None, "RISK_SCORE", "MAX", None)

id,value
0,a Layer object
1,PointWeight_MAX2


In [2]:
arcpy.ba.RemoveSuitabilityCriteria("Suit_Analysis3", "populationtotals.popgrwcyfy;householdincome.pci_cy;policy.white_cy;policy.black_cy;policy.amerind_cy;policy.asian_cy;policy.pacific_cy;policy.othrace_cy;policy.race2up_cy")

<Result 'Suit_Analysis3'>

In [ ]:
#I tried to set the criteria properties but it didn't work for any of the variables whether they came from Esri or elsewhere
#the error message was vague and I never figured out what wasn't working
#a bunch of error messages:

In [6]:
#set properties for demographic data
arcpy.ba.SetCriteriaProperties("Suit_Analysis3", "populationtotals.popgrwcyfy 'Criterion for populationtotals.popgrwcyfy variable' 1 # # POSITIVE # true; householdincome.pci_cy 'Criterion for householdincome.pci_cy variable' 2 # # INVERSE # true; policy.white_cy 'Criterion for policy.white_cy variable' 0 # # POSITIVE # true; policy.black_cy 'Criterion for policy.black_cy variable' 1 # # POSITIVE # true; policy.amerind_cy 'Criterion for policy.amerind_cy variable' 1 # # POSITIVE # true; policy.asian_cy 'Criterion for policy.asian_cy variable' 1 # # POSITIVE # true; policy.pacific_cy 'Criterion for policy.pacific_cy variable' 1 # # POSITIVE # true; policy.othrace_cy 'Criterion for policy.othrace_cy variable' 1 # # POSITIVE # true; policy.race2up_cy 'Criterion for policy.race2up_cy variable' 1 # # POSITIVE # true")

RuntimeError: Object: Error in executing tool

In [14]:
#set properties for NRI Risk data
arcpy.ba.SetCriteriaProperties("Suit_Analysis3", "PointWeight_Max2 'RISK_SCORE' 2 # # POSITIVE # true")

RuntimeError: Object: Error in executing tool

In [1]:
#set properties for solar data
arcpy.ba.SetCriteriaProperties("Suit_Analysis3", "solar_potential 'ave_dni' 1 # # POSITIVE # true")

RuntimeError: Object: Error in executing tool

In [ ]:
#set properties for geothermal data
arcpy.ba.SetCriteriaProperties("Suit_Analysis3", "geo_potential 'class' 1 # # POSITIVE # true")

In [ ]:
#set properties for wind data
arcpy.ba.SetCriteriaProperties("Suit_Analysis3", "wind_potential 'a30' 1 # # POSITIVE # true")

In [15]:
#however, I was still able to successfully run the final calculation
#because the criteria was set at default amounts when I originally set the variables
#but this output is definitely wrong, because I was never able to set important stipulations, like setting Income as 'Inverse'
#so I have even less insight into the weighting than I did in my trial run in the GUI
#the final output used in my lab report came from running the data formatted above in the GUI tools

arcpy.ba.CalculateSuitabilityScore("Suit_Analysis3")

<Result 'Suit_Analysis3'>

## Format layout

In [18]:
#format and export
aprx = arcpy.mp.ArcGISProject(r"C:\Users\celia\Documents\ArcGIS\Projects\GIS5572_FinalProj\GIS5572_FinalProj.aprx")
lyr = aprx.listLayouts("Final_Layout")[0]
lyr.exportToPDF(r"C:\Users\celia\Documents\ArcGIS\Projects\GIS5572_FinalProj\Final_Layout.pdf", resolution = 300)

'C:\\Users\\celia\\Documents\\ArcGIS\\Projects\\GIS5572_FinalProj\\Final_Layout.pdf'